# CBSE Data

In [84]:
#Importing the modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import lxml
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [85]:
#Getting the Main Data
url = "https://www.thelearningpoint.net/home/crawledinformation/a-list-of-cbse-schools-in-india/a-list-of-cbse-schools-in-delhi"
html = requests.get(url).text
soup = BeautifulSoup(html, "lxml")
main = soup.find("td", class_ = "sites-layout-tile sites-tile-name-content-1")
content = main.find_all("table")

df = pd.DataFrame(columns = ["name","address","phone","web","principal"])
l = []

for con in content:
    rows = con.find_all("tr")
    for row in rows:
        tds = row.find_all("td")
        for td in tds:
            l.append(td.text)
        df = df.append({"name":l[0],"address":l[1],"phone":l[2],"web":l[3],"principal":l[4]}, ignore_index=True)
        l = []
df.to_excel(r"C:\Users\jaske\OneDrive\Desktop\cbse.xlsx", index = False)

KeyboardInterrupt: 

In [ ]:
#Getting the Protected Emails
url = "https://www.thelearningpoint.net/home/crawledinformation/a-list-of-cbse-schools-in-india/a-list-of-cbse-schools-in-delhi"
chromedriver = r"C:\Users\jaske\Downloads\chromedriver_win32\chromedriver"
driver = webdriver.Chrome(chromedriver)
driver.get(url)

df = pd.DataFrame(columns = ["email"])
l = []

body = driver.find_element_by_xpath('//*[@id="sites-canvas-main-content"]/table/tbody/tr/td')
tables = body.find_elements_by_tag_name("table")
for tab in tables:
    trs = tab.find_elements_by_tag_name("tr")
    for tr in trs:
        tds = tr.find_elements_by_tag_name("td")
        count = 0
        for td in tds:
            count = count + 1
            if(count == 4):
                df = df.append({"email":td.text}, ignore_index=True)
driver.close()

In [ ]:
#Manipulating the Data
df = pd.read_excel(r"C:\Users\jaske\OneDrive\Desktop\cbse2.xlsx")

df[["emails","website"]] = df["email"].str.split("\n,", expand = True)
df["affiliation number"] = df["name"].str[-7:]
df["name"] = df["name"].str[:-7]
df = df.drop(["email"], axis = 1)

df.to_excel(r"C:\Users\jaske\OneDrive\Desktop\cbse_final.xlsx", index = False)

# Data

In [25]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import lxml
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [26]:
url = "http://cbseaff.nic.in/cbse_aff/schdir_Report/userview.aspx"
chromedriver = r"C:\Users\jaske\Downloads\chromedriver_win32\chromedriver"
driver = webdriver.Chrome(chromedriver)
driver.get(url)

radio = driver.find_element_by_xpath('//*[@id="optlist_2"]')
radio.click()

state = Select(driver.find_element_by_xpath('//*[@id="ddlitem"]'))
state.select_by_value("21")

#district = Select(driver.find_element_by_xpath('//*[@id="DropDownDistrict"]'))
#district.select_by_value("30")

search = driver.find_element_by_xpath('//*[@id="search"]')
search.click()

num = driver.find_element_by_xpath('//*[@id="lbltotal1"]').text

In [30]:
for i in range(1, int(num)//25 + 2):
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")
    content = soup.find("table", {"id":"T1"})
    main = content.find("tbody")
    tables = main.find_all("table")
    count = 0
    desktop = r"C:\Users\jaske\OneDrive\Desktop"
    path1 = desktop + "\data1.csv"
    path2 = desktop + "\data2.csv"
    for tab in tables[:][1:]:
        l = []
        count = count + 1
        tds = tab.find_all("td")
        for td in tds:
            l.append(td.text.replace("\t","").replace("\n","").replace(",","\\"))
        if(count%3 == 1):
            pass
        else:
            if(count%3 == 0):
                with open(path2, "a", newline = "") as fp:
                    wr = csv.writer(fp, dialect='excel')
                    wr.writerow(l)
                fp.close()
            elif((count%2 == 0) or (count%2 == 1)):
                with open(path1, "a", newline = "") as f:
                    wr = csv.writer(f, dialect='excel')
                    wr.writerow(l)
                f.close()
                
    try:    
        nextbut = WebDriverWait(driver,30).until(EC.presence_of_element_located((By.XPATH,'//*[@id="Button1"]')))
        driver.execute_script("arguments[0].click();", nextbut)
    except Exception as e:
        pass
driver.close()
f.close()
fp.close()

df1 = pd.read_csv(path1, encoding='cp1252', header = None)
df2 = pd.read_csv(path2, encoding='cp1252', header = None)
df1.insert(0, 'sr', range(0, len(df1)))
df2.insert(0, 'sr', range(0, len(df2)))

df3 = pd.merge(df1,df2, on=['sr'])
df3.to_excel(desktop + "\\" + "maindata.xlsx")